In [1]:
import torch
import torch.nn as nn
import numpy as np



In [ ]:

alpha = 0.1


def fourier_series(n):
    # n even
    return -480/(np.pi**4*(n**4))


def heat_function(x, t: int):
    a_0 = 1/3
    sum = 0
    for i in range(1, 20):
        exponential = np.exp(-1*alpha*(2*i*np.pi)**2*t)
        sum += fourier_series(2*i)*np.cos(np.pi*2*i*x)*exponential
    return a_0 + sum


def compute_residual(model, x, t):

    x = x.clone().detach().requires_grad_(True)
    t = t.clone().detach().requires_grad_(True)

    u = model(x, t)
    ones = torch.ones_like(u)
    d_t = torch.autograd.grad(
        u, t, grad_outputs=ones, create_graph=True, retain_graph=True
        )[0]
    d_x = torch.autograd.grad(
        u, x, grad_outputs=ones, create_graph=True, retain_graph=True
    )[0]
    d_xx = torch.autograd.grad(
        d_x, x, grad_outputs=torch.ones_like(d_x), create_graph=True,
        retain_graph=True
    )[0]

    return d_t - alpha*d_xx


def initial_condition(x):
    return 10*(x-x**2)**2 + 3


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        layer = [nn.Linear(2, 100), nn.Tanh()]
        for i in range(8):
            layer += [nn.Linear(100, 100), nn.Tanh()]
        layer += [nn.Linear(100, 1), nn.Tanh()]
        self.net = nn.Sequential(*layer)

    def forward(self, x, t):
        inp = torch.cat([x, t], dim=1)
        return self.net(inp)


In [5]:
num_epochs=5000
num_collocation_res=1000
num_collocation_ic=500
num_collocation_bc=600
lr=1e-3
lambda_residual=10.0
lambda_ic=6.0
lambda_bc=5.0




x_col_res = torch.empty(num_collocation_res, 1).uniform_(0, 1)
t_col_res = torch.empty(num_collocation_res, 1).uniform_(0, 1)


In [7]:
x_col_res = torch.rand(num_collocation_res, 1)
t_col_res = torch.rand(num_collocation_res, 1)

print(x_col_res)

tensor([[3.9953e-01],
        [8.5663e-01],
        [9.1472e-01],
        [9.3553e-01],
        [8.0273e-01],
        [6.5660e-01],
        [2.7871e-01],
        [9.9199e-01],
        [4.0111e-01],
        [3.6810e-01],
        [8.9873e-02],
        [3.8951e-02],
        [7.0277e-01],
        [2.8501e-01],
        [2.4977e-01],
        [5.6324e-01],
        [5.4993e-01],
        [7.8503e-01],
        [8.3449e-02],
        [1.6687e-01],
        [8.2406e-02],
        [1.0777e-01],
        [8.5419e-01],
        [1.4175e-01],
        [3.3137e-01],
        [2.7626e-02],
        [4.9483e-01],
        [8.2423e-02],
        [9.3829e-01],
        [1.7282e-01],
        [7.1924e-01],
        [1.1763e-01],
        [3.5793e-01],
        [1.0109e-01],
        [8.0584e-01],
        [7.9129e-01],
        [9.2481e-01],
        [6.5355e-01],
        [5.6736e-01],
        [8.2134e-01],
        [4.2628e-01],
        [4.6012e-01],
        [5.1765e-01],
        [8.6533e-01],
        [9.5279e-01],
        [9

In [ ]:


def train_pinn(
        num_epochs=5000,
        num_collocation_res=1000,
        num_collocation_ic=500,
        num_collocation_bc=600,
        lr=1e-3,
        lambda_residual=10.0,
        lambda_ic=6.0,
        lambda_bc=5.0
):
    model = NeuralNetwork()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Residual Collocation
    x_col_res = torch.empty(num_collocation_res, 1).uniform_(0, 1)
    t_col_res = torch.empty(num_collocation_res, 1).uniform_(0, 1)

    # Initial Condition Collocation
    x_col_ic = torch.empty(num_collocation_ic, 1).uniform_(0, 1)
    t_col_ic = torch.zeros((num_collocation_ic, 1))

    # Boundary Condition Collocation
    t_x_bc = torch.empty(num_collocation_bc, 1).uniform_(0, 1)
    x_bc = torch.zeros((num_collocation_bc, 1), requires_grad=True)
    t_l_bc = torch.empty(num_collocation_bc, 1).uniform_(0, 1)
    l_bc = torch.ones((num_collocation_bc, 1), requires_grad=True)

    # Neumann
    ux_0_bc = torch.zeros((num_collocation_bc, 1))
    ux_1_bc = torch.zeros((num_collocation_bc, 1))

    for _ in range(num_epochs):
        optimizer.zero_grad()

        # Residual
        residual = compute_residual(model, x_col_res, t_col_res)
        loss_residual = torch.mean(residual**2)

        # Initial
        model_ic = model(x_col_ic, t_col_ic)
        loss_ic = torch.mean((model_ic-initial_condition(x_col_ic))**2)

        # Boundary
        u_0_bc = model(x_bc, t_x_bc)
        du_0_bc = torch.autograd.grad(
            u_0_bc, x_bc, grad_outputs=torch.ones_like(u_0_bc),
            create_graph=True
        )[0]

        u_l_bc = model(l_bc, t_l_bc)
        du_l_bc = torch.autograd.grad(
            u_l_bc, l_bc, grad_outputs=torch.ones_like(u_l_bc),
            create_graph=True
        )[0]

        loss_0_bc = torch.mean((du_0_bc-ux_0_bc)**2)
        loss_1_bc = torch.mean((du_l_bc-ux_1_bc)**2)
        loss_b = (loss_0_bc + loss_1_bc)
        loss = lambda_residual*loss_residual+lambda_ic*loss_ic+lambda_bc*loss_b
        loss.backward()
        optimizer.step()

        if _ % 200 == 0:
            print(loss)
    save_path = "parametersheat.pth"
    torch.save(
            {'model_state_dict': model.state_dict()}, save_path
        )
    return model


In [ ]:


model = train_pinn()
